# Exercício Prático 02.4 - Multiplos Objetos

- **Author:** Gabriel Van Loon
- **Data:** Abril/2021
- **Objetivo:** Faça um programa que desenhe dois objetos diferentes na tela (exemplo: triângulo e quadrado). Aplique uma transformação diferente para cada objeto.
   - **Circulo:** AWSD move o objeto
   - **Triangulo:** AS realizam upscaling, AD realizam a rotacao

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

In [2]:
WINDOW_WIDTH = 600
WINDOW_HEIGHT = 600
vertices = []

Declarando os uniforms utilizados nas rotações

In [3]:
uniform_upscale   = 1.0
uniform_rotate    = 0.0
uniform_translate = [0.0, 0.0] 

### Funções para definição de Janelas

Encapsula as funções que criam a única e principal janela do programa.

In [4]:
def configure_window():
    glfw.init()
    glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
    glfw.window_hint(glfw.RESIZABLE, glfw.FALSE)
    window = glfw.create_window(WINDOW_WIDTH, WINDOW_HEIGHT, "Computer Graphics 101", None, None)
    glfw.make_context_current(window)
    return window

# Starting window
window = configure_window()

### Classe para Criação de Shaders

Encapsula as funções que permitem criar e utilizar um determinado shader

In [5]:
class Shader:
    def __init__(self, vertex_code = "", fragment_code = ""):
        self.program = glCreateProgram()
        
        # Create the vertex and shader program
        vertex   = glCreateShader(GL_VERTEX_SHADER)
        fragment = glCreateShader(GL_FRAGMENT_SHADER)
        
        # Set shaders sources code
        glShaderSource(vertex, vertex_code)
        glShaderSource(fragment, fragment_code)
        
        # Compiling vertex shader
        glCompileShader(vertex)
        if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
            error = glGetShaderInfoLog(vertex).decode()
            print(error)
            raise RuntimeError("Erro de compilacao do Vertex Shader")
        
        # Compile fragment shader
        glCompileShader(fragment)
        if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
            error = glGetShaderInfoLog(fragment).decode()
            print(error)
            raise RuntimeError("Erro de compilacao do Fragment Shader")
            
        # If success atach the compiled codes to the program
        glAttachShader(self.program, vertex)
        glAttachShader(self.program, fragment)
        
        # Build program
        glLinkProgram(self.program)
        if not glGetProgramiv(self.program, GL_LINK_STATUS):
            print(glGetProgramInfoLog(self.program))
            raise RuntimeError('Linking error')
            
        # Delete shaders (we don't need them anymore after compile)
        glDeleteShader(vertex);
        glDeleteShader(fragment)
    
    def use(self):
        glUseProgram(self.program)
        
        # Define the attributes mapping from buffer (stride = vertices.strides[0] = 12)
        attr_loc = glGetAttribLocation(self.program, "position")
        glEnableVertexAttribArray(attr_loc)
        glVertexAttribPointer(attr_loc, 3, GL_FLOAT, False, 12, ctypes.c_void_p(0))
        
        # Define the uniforms in the shader
        loc = glGetUniformLocation(self.program, "u_upscale");
        glUniform1f(loc, uniform_upscale);
        
        loc = glGetUniformLocation(self.program, "u_rotate");
        glUniform1f(loc, uniform_rotate);
        
        loc = glGetUniformLocation(self.program, "u_translate");
        glUniform2f(loc, uniform_translate[0], uniform_translate[1]);
        
    def setFloat(self, name, value):
        loc = glGetAttribLocation(program, name)
        glUniform1f(loc, value)

### Shader 1 - Círculo

In [6]:
vertex_code = """
    attribute vec3 position;
    varying   vec3 fPosition;
    
    uniform float u_upscale;
    uniform float u_rotate;
    uniform vec2  u_translate;
    
    void main(){ 
       // Upscaling 
       mat4 upscale_matrix = mat4(
           u_upscale, 0.0, 0.0, 0.0,   // first column
           0.0, u_upscale, 0.0, 0.0,   // second column        
           0.0, 0.0, u_upscale, 0.0,   
           0.0, 0.0, 0.0,       1.0
       );
       
       // Rotate
       mat4 rotate_matrix = mat4(
           cos(u_rotate),  sin(u_rotate), 0.0, 0.0,   // first column
           -sin(u_rotate), cos(u_rotate), 0.0, 0.0,   // second column
           0.0, 0.0, 1.0, 0.0,                        // last column
           0.0, 0.0, 0.0, 1.0
       );
       
       // Translate
       mat4 translate_matrix = mat4(
           1.0, 0.0, 0.0, 0.0,   
           0.0, 1.0, 0.0, 0.0,
           0.0, 0.0, 1.0, 0.0,
           u_translate.x, u_translate.y, 1.0, 1.0
       );
       
       // Aplying transformations (Upscale + Rotate + Translate)
       vec4 coord =  vec4(position.xy, 0.0, 1.0);
       // coord      =  upscale_matrix * coord;
       // coord      =  rotate_matrix * coord;
       coord      =  translate_matrix * coord;
    
       gl_Position = coord;
       fPosition   = gl_Position.xyz;
    }
"""

frag_code = """
    void main(){
        gl_FragColor = vec4(1.0, 0.0, 0.0, 0.5);
    }
"""

circle_shader = Shader(vertex_code, frag_code)

In [7]:
def generate_circle_vertexes(N=64, center=(0,0), radius=1.0):
    circle_points = []
    for i in range(0, N): 
        x = center[0] + radius * np.cos(2*i*np.pi/N)
        y = center[1] + radius * np.sin(2*i*np.pi/N)
        circle_points += [(x, y, 0.0)]
    return circle_points

circle_offset       = len(vertices)
circle_qtd_vertices = 64
vertices += generate_circle_vertexes(circle_qtd_vertices, center=(0.0,0.0), radius=0.60)

In [8]:
def draw_circle(vertices):
    # Select the shader
    circle_shader.use()
    
    # Draw objects
    glDrawArrays(GL_TRIANGLE_FAN, circle_offset, circle_qtd_vertices)

### Shader 2 - Triangle

In [9]:
vertex_code = """
    attribute vec3 position;
    varying   vec3 fPosition;
    
    uniform float u_upscale;
    uniform float u_rotate;
    uniform vec2  u_translate;
    
    void main(){ 
       // Upscaling 
       mat4 upscale_matrix = mat4(
           u_upscale, 0.0, 0.0, 0.0,   // first column
           0.0, u_upscale, 0.0, 0.0,   // second column        
           0.0, 0.0, u_upscale, 0.0,   
           0.0, 0.0, 0.0,       1.0
       );
       
       // Rotate
       mat4 rotate_matrix = mat4(
           cos(u_rotate),  sin(u_rotate), 0.0, 0.0,   // first column
           -sin(u_rotate), cos(u_rotate), 0.0, 0.0,   // second column
           0.0, 0.0, 1.0, 0.0,                        // last column
           0.0, 0.0, 0.0, 1.0
       );
       
       // Translate
       mat4 translate_matrix = mat4(
           1.0, 0.0, 0.0, 0.0,   
           0.0, 1.0, 0.0, 0.0,
           0.0, 0.0, 1.0, 0.0,
           u_translate.x, u_translate.y, 1.0, 1.0
       );
       
       // Aplying transformations (Upscale + Rotate + Translate)
       vec4 coord =  vec4(position.xy, 0.0, 1.0);
       coord      =  upscale_matrix * coord;
       coord      =  rotate_matrix * coord;
       // coord      =  translate_matrix * coord;
    
       gl_Position = coord;
       fPosition   = gl_Position.xyz;
    }
"""

frag_code = """
    void main(){
        gl_FragColor = vec4(0.0, 0.0, 1.0, 0.5);
    }
"""

triangle_shader = Shader(vertex_code, frag_code)

In [10]:
triangle_offset       = len(vertices)
triangle_qtd_vertices = 3
vertices += generate_circle_vertexes(triangle_qtd_vertices, center=(0.0,0.0), radius=0.30)

def draw_triangle(vertices):
    # Select the shader
    triangle_shader.use()
    
    # Draw objects
    glDrawArrays(GL_TRIANGLE_FAN, triangle_offset, triangle_qtd_vertices)

### Capturando Eventos do Teclado e Mouse

- **Mouse button references:** https://www.glfw.org/docs/latest/group__buttons.html
- **Keyboard button references:** https://www.glfw.org/docs/latest/group__keys.html

In [11]:
# uniform_upscale   = 0.0
# uniform_rotate    = 0.0
# uniform_translate = [0.0, 0.0] 

def key_event(window, key, scancode, action, mods):
    print('[key event] key=',key)
    print('[key event] scancode=',scancode)
    print('[key event] action=',action)
    print('[key event] mods=',mods)
    print('-------')
    global uniform_upscale, uniform_rotate, uniform_translate
    
    if key == glfw.KEY_W and action == 1:
        uniform_translate[1] += 0.05
        uniform_upscale      += 0.05
    if key == glfw.KEY_S and action == 1:
        uniform_translate[1] -= 0.05
        uniform_upscale      -= 0.05
        
    if key == glfw.KEY_A and action == 1:
        uniform_translate[0] -= 0.05
        uniform_rotate       -= 0.05
    if key == glfw.KEY_D and action == 1:
        uniform_translate[0] += 0.05
        uniform_rotate      += 0.05
        
    # return False

def mouse_event(window, button, action, mods):
#     print('[mouse event] button=',button)
#     print('[mouse event] action=',action)
#     print('[mouse event] mods=',mods)
#     print('-------')
    return False
    
glfw.set_key_callback(window, key_event)
glfw.set_mouse_button_callback(window, mouse_event)

### Iniciando o Programa e suas variáveis

A primeira coisa que precisamos fazer é iniciar o buffer e enviar os vértices para a GPU

In [12]:
# Convertendo para numpy array
vertices = np.array(vertices, dtype=np.float32)

In [13]:
# Gerando um Buffer para o Array de vértices
buffer = glGenBuffers(1)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Enviando os dados definindo o buffer como estatico, pois não pretendemos a
# posição dos vértices iniciais (GL_DYNAMIC_DRAW | GL_STATIC_DRAW)
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)

In [14]:
glfw.show_window(window)
while not glfw.window_should_close(window):
    glfw.poll_events() 
    
    # Reset the screen with the white color
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    # Draw objects
    draw_circle(vertices)
    draw_triangle(vertices)
    
    glfw.swap_buffers(window)
glfw.terminate()

[key event] key= 262
[key event] scancode= 114
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 262
[key event] scancode= 114
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 262
[key event] scancode= 114
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 262
[key event] scancode= 114
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 262
[key event] scancode= 114
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 262
[key event] scancode= 114
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 262
[key event] scancode= 114
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 262
[key event] scancode= 114
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 262
[key event] scancode= 114
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 262
[key event] scancode= 114
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 263
[key even

[key event] key= 83
[key event] scancode= 39
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 83
[key event] scancode= 39
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 83
[key event] scancode= 39
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 83
[key event] scancode= 39
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 83
[key event] scancode= 39
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 83
[key event] scancode= 39
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 83
[key event] scancode= 39
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 83
[key event] scancode= 39
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 83
[key event] scancode= 39
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 83
[key event] scancode= 39
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 83
[key event] scancode= 39
[key 

[key event] key= 87
[key event] scancode= 25
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 87
[key event] scancode= 25
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 87
[key event] scancode= 25
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 87
[key event] scancode= 25
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 87
[key event] scancode= 25
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 87
[key event] scancode= 25
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 87
[key event] scancode= 25
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 87
[key event] scancode= 25
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 87
[key event] scancode= 25
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 87
[key event] scancode= 25
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 87
[key event] scancode= 25
[key 

[key event] key= 83
[key event] scancode= 39
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 83
[key event] scancode= 39
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 83
[key event] scancode= 39
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 68
[key event] scancode= 40
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 68
[key event] scancode= 40
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 83
[key event] scancode= 39
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 83
[key event] scancode= 39
[key event] action= 0
[key event] mods= 0
-------
[key event] key= 87
[key event] scancode= 25
[key event] action= 1
[key event] mods= 0
-------
[key event] key= 87
[key event] scancode= 25
[key event] action= 0
[key event] mods= 0
-------


### Referencias de matrizes

A seguir apenas um referencial de como montar as matrizes de transformação

#### a) Matriz de Translação 2D (z = 0)

$$
\begin{pmatrix}
x_f \\ y_f \\ z_f \\ h_f \\
\end{pmatrix} 
= 
\begin{pmatrix}
1 & 0 & 0 & t_x \\
0 & 1 & 0 & t_y \\
0 & 0 & t_z & 0\\
0 & 0 & 0 &1 \\
\end{pmatrix} 
*
\begin{pmatrix}
x_0 \\ y_0 \\ z_0 \\ h_0 \\
\end{pmatrix} 
$$

#### b) Matriz de Escala 2D (z=0)

$$
\begin{pmatrix}
x_f \\ y_f \\ z_f \\ 1
\end{pmatrix} 
= 
\begin{pmatrix}
s_x & 0 & 0 & 0  \\
0   & s_y & 0 & 0\\
0   & 0 & s_z & 0\\
0   & 0 & 0 & 1 \\
\end{pmatrix} 
*
\begin{pmatrix}
x_0 \\ y_0 \\ z_0 \\ 1
\end{pmatrix} 
$$

#### c) Matriz de Rotação 2D (z=0)

$$
\begin{pmatrix}
x_f \\ y_f \\ z_f \\ 1
\end{pmatrix} 
= 
\begin{pmatrix}
\cos(\theta) & -\sin(\theta) & 0 & 0\\
\sin(\theta) & \cos(\theta) & 0  & 0\\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{pmatrix} 
*
\begin{pmatrix}
x_0 \\ y_0 \\ z_0 \\ 1
\end{pmatrix} 
$$